This notebook tends to offer a possible optimization setting where naive Bayesian optimization would fail

The main idea is to consider a signal which is not iid (e.g. when the signal has associated seasonality and the optimizaion domain is essentially 2d)

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time
%matplotlib inline

In [ ]:
x = np.linspace(0, 1, 1000)
plt.plot(x, np.sin(2 * math.pi * np.linspace(-0 * 1.0 / 7, 1 - 0 * 1.0 / 7, 1000)), label="0")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-1 * 1.0 / 7, 1 - 1 * 1.0 / 7, 1000)), label="1")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-2 * 1.0 / 7, 1 - 2 * 1.0 / 7, 1000)), label="2")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-3 * 1.0 / 7, 1 - 3 * 1.0 / 7, 1000)), label="3")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-4 * 1.0 / 7, 1 - 4 * 1.0 / 7, 1000)), label="4")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-5 * 1.0 / 7, 1 - 5 * 1.0 / 7, 1000)), label="5")
plt.plot(x, np.sin(2 * math.pi * np.linspace(-6 * 1.0 / 7, 1 - 6 * 1.0 / 7, 1000)), label="6")
plt.title("Example of a moving periodic signal")
plt.legend()
plt.show()

In [ ]:
class DataGenerator:
    def __init__(self, sigma_obs):
        self.domain = np.linspace(0, 1, 1000)
        self.day_of_the_week = 0
        self.true_y = {i : np.sin(2 * math.pi * np.linspace(-i * 1.0 / 7, 1 - i * 1.0 / 7, 1000)) for i in range(7)}
        self.sigma_obs = sigma_obs
        self.true_y_dict = {i: {x: y for (x, y) in zip(self.domain, self.true_y[i])} for i in range(7)}

    def sample(self, x):
        # Handling cases when x is both scalar and numpy array
        self.day_of_the_week = (self.day_of_the_week + 1) % 7
        if type(x) == np.ndarray:
            return np.array(map(lambda z: self.true_y_dict[self.day_of_the_week][z], x)) + self.sigma_obs * np.random.randn(x.shape[0])
        else:
            return self.true_y_dict[self.day_of_the_week][x] + self.sigma_obs * np.random.randn()


In [ ]:
import sys
sys.path.append("..")
from bayes_opt import BayesOpt

MCMC_OPTS = {"prior": lambda l: int(l > 0 and l < 1),
             "icdf": lambda l: l,
             "jump": lambda l: l + 0.05 * np.random.randn(),
             "burn_period": 10000,
             "mcmc_samples": 25}
data_gen = DataGenerator(sigma_obs=0.1)

opt_engine = BayesOpt(data_gen, init_sample_size=1, max_steps=50, sigma_obs=0.1, 
                      is_mcmc=True, mcmc_opts=MCMC_OPTS)

for _ in range(50):
    opt_engine.step()
    
    true_y = data_gen.true_y[data_gen.day_of_the_week]
    sample_x = opt_engine.x
    sample_y = opt_engine.y
    mu = opt_engine.mu_posterior
    std_1d = opt_engine.std_posterior

    plt.plot(x, true_y, label="true_signal")
    plt.plot(sample_x, sample_y, ".", color="r", label="picked_x")
    plt.plot([sample_x[-1]], [sample_y[-1]], ".", color="b", label="last_x")
    plt.plot(x, mu, color="g", label="posterior")
    plt.fill_between(x, mu - 2 * std_1d, mu + 2 * std_1d, color="g", alpha=0.5)

    plt.title("True and recovered signals")
    plt.legend()
    plt.xlabel("x")
    plt.show()

    display.display(plt.gcf())
    display.clear_output(wait=True)
    time.sleep(2)    